
#import modules








In [35]:
import pandas as pd
import requests


#get data from wiki
#obtain tables 
#get the first table

In [36]:
web=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
list_web = pd.read_html(web)
df = list_web[0]

#assign first row as column names

In [37]:
df.columns=['Postcode','Borough','Neighbourhood']
df.drop(axis=0,index=0,inplace=True)

#remove rows with 'not assigned'

In [38]:
df_a=df[df['Borough']!='Not assigned']
df_a.reset_index(drop=True, inplace=True)

#merge naborhood with the same postcode
#for unassigned Neighbourhood, use Borough as Neighbourhood 

In [39]:
df_b=df_a.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_b['Neighbourhood'][df_b['Neighbourhood']=='Not assigned']= df_b['Borough'][df_b['Neighbourhood']=='Not assigned']


#check df

In [40]:
df_b.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
df_b.shape

(103, 3)

## step2

In [48]:
geo_code=pd.read_csv('http://cocl.us/Geospatial_data')

In [50]:
geo_code.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [60]:
geo_code.index=geo_code['Postal Code']
geo_code_dict= geo_code.to_dict('index')


43.806686299999996

In [61]:
Latitude=[]
Longitude=[]
for i in range(len(df_b)):
    Lat=geo_code_dict.get(df_b['Postcode'][i]).get('Latitude')
    Lon=geo_code_dict.get(df_b['Postcode'][i]).get('Longitude')
    Latitude.append(Lat)
    Longitude.append(Lon)


In [65]:
df_b['Latitude']=Latitude
df_b['Longitude']=Longitude
df_b.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## step3

#segment the neighborhoods in Borough

In [67]:
df_Scarborough=df_b[df_b['Borough']=='Scarborough']

In [73]:
df_Scarborough[['Latitude','Longitude']].mean()

Latitude     43.766229
Longitude   -79.249085
dtype: float64

In [76]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.70 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  24.92 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  27.06 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00   7.64 MB/s


In [82]:
map_Scarborough = folium.Map(location=[43.766229, -79.249085], zoom_start=12)
for lat, lng, label in zip(df_Scarborough['Latitude'], df_Scarborough['Longitude'], df_Scarborough['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough